In [106]:
import time
import google.generativeai as genai
import os
import typing_extensions as typing
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import ast

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
# Upload the video and print a confirmation.
video_file_name = "test.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")


# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/8wzdr5ky05ln


In [107]:
# Create the prompt.
prompt = f"""
Suppose you are a very liberal stereotypical 18 year old girl watching instagram reels. You
are to watch a video and provide a score rating between 0 and 1, where 
the score is completely based on your prescribed identity's preferences. A 0 indicates
that the video is terrible and you would never want to even look at it, and a 
1 indicates that you are really invested and want to watch the whole thing. 
Try to avoid giving a full 0 or a full 1, as you should have some nuance in your answer.
Provide an explanation for your rating, and give an explanation of the video
"""

class Video(typing.TypedDict):
    rating: float
    rating_reasoning: str
    description: str

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

In [108]:
# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content(
    [video_file, prompt],
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema=Video
    ),
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    }
)

# Print the response, rendering any Markdown
ast.literal_eval(response.text)

Making LLM inference request...


{'description': "Rudy Giuliani is talking about some product, and it's a little funny how he's doing it,  but  the product is kind of lame and a bit too old man",
 'rating': 0.6,
 'rating_reasoning': "The video is funny but also a little sad.  It's funny because it seems like it could be a SNL skit but also sad because it's Rudy Giuliani.  I don't really care about the product, but it's not a bad video to watch."}

In [109]:
# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content(
    [video_file, prompt],
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    }
)

# Print the response, rendering any Markdown
response.text

Making LLM inference request...


'0.3. This is the cringiest thing I\'ve ever seen and makes me want to crawl out of my skin. The video is Rudy Giuliani talking about his "America\'s Mayor" brand of hand sanitizer. He says something about it being "incredibly proud of," and he\'s holding a bottle of hand sanitizer.  He\'s wearing a navy blue suit and a white shirt. \nHe\'s trying to look presidential and important but it\'s just ridiculous. The only reason I\'m giving it a 0.3 is because it\'s so absurd it\'s kind of entertaining. \n'